In [22]:
# These are all the imports that will be needed for the project
import nltk
import time
import random
import re
import mariadb

from nltk.corpus import words as nltkwords
nltk.download('words')

[nltk_data] Downloading package words to C:\Users\Luka
[nltk_data]     Brennan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [23]:
# Database connection, making a connection to the WordGame database
def DatabaseConnection():
    connection = mariadb.connect(
        host = 'localhost',
        user = 'root',
        password = '12July98',
        port = 3308,
        database = 'WordGame'
    )
    return connection

In [24]:
# Generate a random word using the nltk library
from nltk.corpus import words as nltk_words
# Make sure that the word is at least 8 letters long
def GetRandomWord(min_length=8):
    WordList = [word for word in nltk_words.words() if len(word) >= min_length]
    return random.choice(WordList) if WordList else None
RandomWord = GetRandomWord()
print(RandomWord)


carelessness


In [25]:
# Insert the data to the database when the user completes the game
def InsertData(Player, SourceWord, Matches, TimeTaken):
    try:
        connection = DatabaseConnection()
        cursor = connection.cursor()
        cursor.execute(
            "INSERT INTO LeaderBoard (PlayerName, WordGiven, Matches, TimeTaken) VALUES (?, ?, ?, ?)",
            (Player, SourceWord, Matches, TimeTaken)
        )
        connection.commit()
    except mariadb.Error as e:
        print(f"Error inserting data: {e}")
    finally:
        cursor.close()
        connection.close()


In [26]:
# Gets the data from the database and sorts it by TimeTaken
def SortedLeaderBoard():
    connection = DatabaseConnection()
    cursor = connection.cursor()
    cursor.execute(
        "SELECT PlayerName, WordGiven, Matches, TimeTaken FROM LeaderBoard ORDER BY TimeTaken ASC LIMIT 10"
    )
    results = cursor.fetchall()
    return results


In [27]:
# Code checks to make sure that the word entered is valid
def FormWord(word, base_word):
    base_word_list = list(base_word)
    for char in word:
        if char in base_word_list:
            base_word_list.remove(char)
        else:
            return False
    return True
# makes sure that the words used fit the requirments of the rules of the game 
def validate_words(word_list, base_word, min_words_required=7, min_word_length=4):

    valid_words = []
    invalid_letters = set()
    misspelled_words = []
    too_short_words = []
    used_source_word = False
    
    for word in word_list:
        if word == base_word:
            used_source_word = True
        elif len(word) < min_word_length:
            too_short_words.append(word)
        elif word not in nltkwords.words():
            misspelled_words.append(word)
        elif not FormWord(word, base_word):
            invalid_letters.update([char for char in word if char not in base_word])
        elif word in valid_words:
            pass
        else:
            valid_words.append(word)
    
    feedback = []
    
    if invalid_letters:
        feedback.append(f"You used these invalid letters: {', '.join(invalid_letters)}")
    if misspelled_words:
        feedback.append(f"You misspelled these words: {', '.join(misspelled_words)}")
    if too_short_words:
        feedback.append(f"These words are too short: {', '.join(too_short_words)}")
    if used_source_word:
        feedback.append(f"You cannot use the source word: {base_word}")

    if len(valid_words) < min_words_required:
        feedback.append(f"You must provide at least {min_words_required} valid words. Currently, you have {len(valid_words)} valid words.")
    
    if valid_words:
        feedback.append(f"Valid words ({len(valid_words)}): {', '.join(valid_words)}")
    
    return valid_words, '\n'.join(feedback)

In [28]:
# validating each word
def ProcessInput(input_string, base_word):
    word_list = re.split(r'\s+', input_string.strip().lower())
    valid_words, feedback = validate_words(word_list, base_word)
    return valid_words, feedback


In [29]:
def WordGame():
    print(" Welcome To WordGame, Here are the Rules.")
    print("1. You need to enter in at least 7 words with a total of 4 or more letters.")
    print("2. You can not use any illegal characters such as (, . / ; ' @ : > <).")
    print("3. You can not use the SourceWord given as an answer.")
    print("4. You have to use the letters provided in the SourceWord.")
    print("5. There are no duplicate words.")
    print("6. Use a SPACE when typing the next word.")
    print("Note that the game is not case sensitive.")

    SourceWord = RandomWord.lower()
    print(f"Here is your word: {SourceWord}")

    StartTime = time.time()

    while True:
        UserInput = input("Type in the words: ")
        valid_words, results = ProcessInput(UserInput, SourceWord)

        print(results)

        if len(valid_words) >= 7:
            print("All your guesses are valid!")
        else:
            print(f"Game over! You needed at least 7 valid words, but only {len(valid_words)} were valid.")
            return 

        EndTime = time.time()
        time_taken = round(EndTime - StartTime, 2)  # Calculate time taken

        PlayerName = input("Enter your name: ")

        InsertData(PlayerName, SourceWord, ', '.join(valid_words), time_taken)

        print(f"Game Over! Time taken: {time_taken} seconds.")

        # Fetch and display the sorted leaderboard
        leaderboard = SortedLeaderBoard()
        print("\nTop 10 Leaderboard:")
        for position, (name, word, matches, time_taken) in enumerate(leaderboard, start=1):
            print(f"{position}. {name} - {word} - {matches} - {time_taken} seconds")


In [30]:
if __name__ == "__main__":
    WordGame()


 Welcome To WordGame, Here are the Rules.
1. You need to enter in at least 7 words with a total of 4 or more letters.
2. You can not use any illegal characters such as (, . / ; ' @ : > <).
3. You can not use the SourceWord given as an answer.
4. You have to use the letters provided in the SourceWord.
5. There are no duplicate words.
6. Use a SPACE when typing the next word.
Note that the game is not case sensitive.
Here is your word: carelessness
These words are too short: 
You must provide at least 7 valid words. Currently, you have 0 valid words.
Game over! You needed at least 7 valid words, but only 0 were valid.


In [34]:
from flask import Flask, render_template, request, redirect, url_for
import random
import re
import time
import mariadb
# UPDATE for the app.py nltk will not be used as there are many problems with words not being registered,
# as nltk may have some words missing casuing problems in such as this words does not exist when entering in one of the 7 words for the game
import nltk
from nltk.corpus import words as nltkwords

app = Flask(__name__)

nltk.download('words')

def DatabaseConnection():
    connection = mariadb.connect(
        host='localhost',
        user='root',
        password='12July98',
        port=3308,
        database='WordGame'
    )
    return connection

def GetRandomWord(min_length=8):
    WordList = [word for word in nltkwords.words() if len(word) >= min_length]
    return random.choice(WordList) if WordList else None

def InsertData(Player, SourceWord, Matches, TimeTaken):
    try:
        connection = DatabaseConnection()
        cursor = connection.cursor()
        cursor.execute(
            "INSERT INTO LeaderBoard (PlayerName, WordGiven, Matches, TimeTaken) VALUES (?, ?, ?, ?)",
            (Player, SourceWord, Matches, TimeTaken)
        )
        connection.commit()
    except mariadb.Error as e:
        print(f"Error inserting data: {e}")
    finally:
        cursor.close()
        connection.close()

def SortedLeaderBoard():
    connection = DatabaseConnection()
    cursor = connection.cursor()
    cursor.execute(
        "SELECT PlayerName, WordGiven, Matches, TimeTaken FROM LeaderBoard ORDER BY TimeTaken ASC LIMIT 10"
    )
    results = cursor.fetchall()
    return results

def FormWord(word, base_word):
    base_word_list = list(base_word)
    for char in word:
        if char in base_word_list:
            base_word_list.remove(char)
        else:
            return False
    return True

def ValidateWords(word_list, base_word, min_words_required=7, min_word_length=4):
    valid_words = []
    invalid_letters = set()
    misspelled_words = []
    too_short_words = []
    used_source_word = False
    
    for word in word_list:
        if word == base_word:
            used_source_word = True
        elif len(word) < min_word_length:
            too_short_words.append(word)
        elif word not in nltkwords.words():
            misspelled_words.append(word)
        elif not FormWord(word, base_word):
            invalid_letters.update([char for char in word if char not in base_word])
        elif word in valid_words:
            pass 
        else:
            valid_words.append(word)
    
    feedback = []
    
    if invalid_letters:
        feedback.append(f"You used these invalid letters: {', '.join(invalid_letters)}")
    if misspelled_words:
        feedback.append(f"You misspelled these words: {', '.join(misspelled_words)}")
    if too_short_words:
        feedback.append(f"These words are too short: {', '.join(too_short_words)}")
    if used_source_word:
        feedback.append(f"You cannot use the source word: {base_word}")

    if len(valid_words) < min_words_required:
        feedback.append(f"You must provide at least {min_words_required} valid words. Currently, you have {len(valid_words)} valid words.")
    
    if valid_words:
        feedback.append(f"Valid words ({len(valid_words)}): {', '.join(valid_words)}")
    
    return valid_words, '\n'.join(feedback)

def ProcessInput(input_string, base_word):
    word_list = re.split(r'\s+', input_string.strip().lower())
    valid_words, feedback = validate_words(word_list, base_word)
    return valid_words, feedback

@app.route('/')
def menu():
    return render_template('Menu.html')

@app.route('/game', methods=['GET', 'POST'])
def game():
    if request.method == 'POST':
        source_word = request.form['source_word']
        user_input = request.form['user_input']
        start_time = float(request.form['start_time'])
        valid_words, results = ProcessInput(user_input, source_word)
        
        if len(valid_words) >= 7:
            end_time = time.time()
            time_taken = round(end_time - start_time, 2)
            player_name = request.form['player_name']
            InsertData(player_name, source_word, ', '.join(valid_words), time_taken)
            return redirect(url_for('leaderboard'))
        else:
            return render_template('Game.html', source_word=source_word, feedback=results)
    else:
        source_word = GetRandomWord().lower()
        start_time = time.time()
        return render_template('Game.html', source_word=source_word, start_time=start_time)

@app.route('/leaderboard')
def leaderboard():
    leaderboard_data = SortedLeaderBoard()
    return render_template('Leaderboard.html', leaderboard=leaderboard_data)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


[nltk_data] Downloading package words to C:\Users\Luka
[nltk_data]     Brennan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [ ]:
from nltk.corpus import words as nltkwords
# Testing words from nltk, as some of them are not in registered
if "metro" in nltkwords.words():
    print("The word 'metro' is in the NLTK words corpus.")
else:
    print("The word 'metro' is NOT in the NLTK words corpus.")


The word 'metro' is NOT in the NLTK words corpus.


In [ ]:
#pip install requests


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
# Testing oxford api or getting words. 
# |UPDATE| NOT USEING THIS ANYMORE
APP_ID = 'c72fe586'
API_KEY = '891d340e3a07265a0aa0e9be3336fc65'

def validate_word_with_oxford_api(word):
    url = f"https://od-api.oxforddictionaries.com/api/v2/entries/en-us/{word.lower()}"
    headers = {
        "app_id": APP_ID,
        "app_key": API_KEY
    }
    response = requests.get(url, headers=headers)
    return response.status_code == 200  


In [ ]:
#pip install black

^C
Note: you may need to restart the kernel to use updated packages.


SyntaxError: invalid syntax (97274718.py, line 1)